In [9]:
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 65.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [2]:
from transformers import AutoTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
from datasets import Dataset
import pandas as pd

# NSMC 데이터 다운로드
!wget -O ratings_train.txt https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget -O ratings_test.txt https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

train_df = pd.read_table("ratings_train.txt").dropna().drop_duplicates(subset="document")
test_df = pd.read_table("ratings_test.txt").dropna().drop_duplicates(subset="document")

train_dataset = Dataset.from_pandas(train_df[["document", "label"]])
test_dataset = Dataset.from_pandas(test_df[["document", "label"]])

# 전처리
def preprocess(example):
    return tokenizer(example["document"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(preprocess, batched=False)
test_dataset = test_dataset.map(preprocess, batched=False)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

--2025-06-28 08:07:21--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.1s    

2025-06-28 08:07:21 (114 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2025-06-28 08:07:21--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [application/octet-

Map:   0%|          | 0/146182 [00:00<?, ? examples/s]

Map:   0%|          | 0/49157 [00:00<?, ? examples/s]

In [4]:
import torch
from torch.utils.data import Dataset

class KoBERTDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=max_len)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

In [5]:
train_dataset = KoBERTDataset(train_df["document"].tolist(), train_df["label"].tolist(), tokenizer)
test_dataset = KoBERTDataset(test_df["document"].tolist(), test_df["label"].tolist(), tokenizer)

In [6]:
import transformers
print(transformers.__version__)

4.53.0


In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
from transformers import TrainingArguments
help(TrainingArguments)

Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.Sch

In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./kluebert-sentiment",
    num_train_epochs=1,                     # ✅ 에폭 1로
    max_steps=100,                          # ✅ 딱 100스텝만 돌고 종료
    per_device_train_batch_size=8,          # ✅ 배치 줄이면 메모리 아끼고 속도 ↑
    per_device_eval_batch_size=8,
    eval_strategy="no",                     # ✅ eval 생략
    save_strategy="no",                     # ✅ 저장 생략
    logging_strategy="steps",
    logging_steps=20,                       # ✅ 로그도 자주 안 찍게
    report_to="none",
    no_cuda=True                            # ✅ 강제 CPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1577: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Step,Training Loss
20,0.621800
40,0.533500
60,0.543700
80,0.367100
100,0.458200


TrainOutput(global_step=100, training_loss=0.5048779678344727, metrics={'train_runtime': 1260.4869, 'train_samples_per_second': 0.635, 'train_steps_per_second': 0.079, 'total_flos': 52622211072000.0, 'train_loss': 0.5048779678344727, 'epoch': 0.005472555135992995})

In [22]:
model.save_pretrained("kluebert-sentiment")
tokenizer.save_pretrained("kluebert-sentiment")

('kluebert-sentiment/tokenizer_config.json',
 'kluebert-sentiment/special_tokens_map.json',
 'kluebert-sentiment/vocab.txt',
 'kluebert-sentiment/added_tokens.json',
 'kluebert-sentiment/tokenizer.json')

In [23]:
from transformers import pipeline

classifier = pipeline("text-classification", model="kluebert-sentiment", tokenizer="kluebert-sentiment", device=-1)

print(classifier("이 영화 정말 재미없었어요."))  # 예상: 부정
print(classifier("완전 최고였어요!"))          # 예상: 긍정

Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.8056910037994385}]
[{'label': 'LABEL_1', 'score': 0.9818567633628845}]


## Fine-Tuning 이후, 감성 팩터

In [24]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="kluebert-sentiment",
    tokenizer="kluebert-sentiment",
    device=-1  # CPU 사용
)

Device set to use cpu


In [25]:
news_sample = "삼성전자가 사상 최대 실적을 기록했다."
result = classifier(news_sample)
print(result)  # [{'label': 'LABEL_1', 'score': 0.95}] → 긍정

[{'label': 'LABEL_1', 'score': 0.8888140320777893}]


### 뉴스 크롤링

In [18]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=7e50e1efb96a5b5978bfa848f07eb4c0213ef08a6ac86c47a5feb952b9e5eac9
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [19]:
import feedparser
import pandas as pd

def get_naver_rss_news(keyword, max_articles=50):
    url = f"https://news.google.com/rss/search?q={keyword}+site:naver.com&hl=ko&gl=KR&ceid=KR:ko"
    feed = feedparser.parse(url)

    articles = []
    for entry in feed.entries[:max_articles]:
        articles.append({
            "title": entry.title,
            "link": entry.link,
            "published": entry.published
        })
    return pd.DataFrame(articles)

# ✅ 실행
df_rss = get_naver_rss_news("삼성전자", max_articles=30)
print(df_rss.head())
df_rss.to_csv("rss_naver_samsung_news.csv", index=False, encoding="utf-8-sig")

                                               title  \
0  'One UI 8' 정식 출시 임박?…갤럭시 S25 업데이트, 최종 단계 돌입 - ...   
1  삼성전자 갤럭시 S25 256GB 자급제폰 SM-S931N : 삼성모바일파트너 코잇...   
2                         🤎삼성전자 (투자전략) - 네이버 프리미엄콘텐츠   
3           삼성전자배당기준일 : 수급단타왕x만쥬 주식스터디 - 네이버 프리미엄콘텐츠   
4       상승세 끝? 외국인・기관 매도세에 한풀 꺾인 국내 증시 - 네이버페이 머니스토리   

                                                link  \
0  https://news.google.com/rss/articles/CBMihgFBV...   
1  https://news.google.com/rss/articles/CBMibEFVX...   
2  https://news.google.com/rss/articles/CBMigwFBV...   
3  https://news.google.com/rss/articles/CBMi4AFBV...   
4  https://news.google.com/rss/articles/CBMiWEFVX...   

                       published  
0  Fri, 27 Jun 2025 04:14:00 GMT  
1  Thu, 26 Jun 2025 17:47:18 GMT  
2  Fri, 27 Jun 2025 08:14:00 GMT  
3  Thu, 26 Jun 2025 07:23:11 GMT  
4  Thu, 26 Jun 2025 08:35:19 GMT  


In [20]:
import feedparser
import pandas as pd
from datetime import datetime, timedelta

# ✅ 뉴스 수집 함수
def crawl_google_news_rss(keyword, days=7):
    base_url = f"https://news.google.com/rss/search?q={keyword}+when:7d&hl=ko&gl=KR&ceid=KR:ko"
    feed = feedparser.parse(base_url)
    articles = []

    for entry in feed.entries:
        published_dt = datetime(*entry.published_parsed[:6])
        if datetime.now() - published_dt <= timedelta(days=days):
            articles.append({
                "keyword": keyword,
                "title": entry.title,
                "link": entry.link,
                "published": published_dt.strftime("%Y-%m-%d %H:%M:%S")
            })

    return pd.DataFrame(articles)

# ✅ 다수 종목 수집 예시
kospi200_list = ["삼성전자", "LG화학", "NAVER", "POSCO홀딩스", "현대자동차"]  # 필요 시 확장
all_news = []

for keyword in kospi200_list:
    df = crawl_google_news_rss(keyword)
    if not df.empty:
        all_news.append(df)

# ✅ 통합 저장
df_total = pd.concat(all_news, ignore_index=True)
df_total.to_csv("kospi200_news_last7days.csv", index=False, encoding="utf-8-sig")

# ✅ 결과 확인
print(df_total.shape)
print(df_total.head())

(398, 4)
  keyword                                              title  \
0    삼성전자  삼성전자, '프로젝트 무한' 올레도스 공급망 삼성D로 이원화 검토 - 전자부품 전문...   
1    삼성전자                 삼성전자 美 테일러 팹 내년 초 양산라인 투자 - 지디넷코리아   
2    삼성전자            드디어 '삼성전자의 시간' 오나…'가격 급등' 이례적 현상 - 한국경제   
3    삼성전자            “美 정부, 삼성전자·하이닉스 中 공장에 장비 반입 제한” - 조선일보   
4    삼성전자  삼성전자 파운드리 퀄컴 2나노 AP 수주할 듯, 한진만 내년 파운드리 부활 신호탄 ...   

                                                link            published  
0  https://news.google.com/rss/articles/CBMiZkFVX...  2025-06-25 06:36:02  
1  https://news.google.com/rss/articles/CBMiVkFVX...  2025-06-23 05:13:16  
2  https://news.google.com/rss/articles/CBMiWkFVX...  2025-06-22 08:29:49  
3  https://news.google.com/rss/articles/CBMigwFBV...  2025-06-22 16:19:04  
4  https://news.google.com/rss/articles/CBMic0FVX...  2025-06-25 05:58:51  


In [28]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 22.5 MB/s eta 0:00:00


,ticker,corp_name
0,095570,AJ네트웍스
1,006840,AK홀딩스
2,027410,BGF
3,282330,BGF리테일
4,138930,BNK금융지주


In [33]:
from pykrx import stock
import pandas as pd
from datetime import datetime, timedelta
import feedparser
from tqdm import tqdm

# ✅ 기준일과 코스피200 종목 추출
date = "20241227"
code = "1028"  # KOSPI 200 index code
tickers = stock.get_index_portfolio_deposit_file(code, date)

# ✅ 종목명 매핑 → df_kospi200
df_kospi200 = pd.DataFrame({
    "ticker": tickers,
    "corp_name": [stock.get_market_ticker_name(t) for t in tickers]
})

print("✅ 코스피200 구성 종목 수:", len(df_kospi200))
print(df_kospi200.head())

✅ 코스피200 구성 종목 수: 200
   ticker corp_name
0  005930      삼성전자
1  000660    SK하이닉스
2  373220  LG에너지솔루션
3  207940  삼성바이오로직스
4  005380       현대차


In [35]:
from urllib.parse import quote  # 추가

def crawl_google_news_rss(keyword, days=7):
    encoded_keyword = quote(keyword)  # 공백, 특수문자 인코딩
    base_url = f"https://news.google.com/rss/search?q={encoded_keyword}+when:{days}d&hl=ko&gl=KR&ceid=KR:ko"
    feed = feedparser.parse(base_url)
    articles = []

    for entry in feed.entries:
        published_dt = datetime(*entry.published_parsed[:6])
        if datetime.now() - published_dt <= timedelta(days=days):
            articles.append({
                "keyword": keyword,
                "title": entry.title,
                "link": entry.link,
                "published": published_dt.strftime("%Y-%m-%d %H:%M:%S")
            })

    return pd.DataFrame(articles)

# ✅ 전체 코스피200 종목 뉴스 수집
all_news = []
for name in tqdm(df_kospi200["corp_name"]):
    df_news = crawl_google_news_rss(name, days=7)
    if not df_news.empty:
        all_news.append(df_news)

# ✅ 뉴스 통합 저장
df_total_news = pd.concat(all_news, ignore_index=True)
df_total_news.to_csv("kospi200_news_last7days.csv", index=False, encoding="utf-8-sig")

# ✅ 결과 확인
print("✅ 전체 수집된 뉴스 수:", len(df_total_news))
print(df_total_news.head())

100%|██████████| 200/200 [01:05<00:00,  3.04it/s]


✅ 전체 수집된 뉴스 수: 14072
  keyword                                              title  \
0    삼성전자  삼성전자, '프로젝트 무한' 올레도스 공급망 삼성D로 이원화 검토 - 전자부품 전문...   
1    삼성전자                 삼성전자 美 테일러 팹 내년 초 양산라인 투자 - 지디넷코리아   
2    삼성전자            드디어 '삼성전자의 시간' 오나…'가격 급등' 이례적 현상 - 한국경제   
3    삼성전자            “美 정부, 삼성전자·하이닉스 中 공장에 장비 반입 제한” - 조선일보   
4    삼성전자  삼성전자 파운드리 퀄컴 2나노 AP 수주할 듯, 한진만 내년 파운드리 부활 신호탄 ...   

                                                link            published  
0  https://news.google.com/rss/articles/CBMiZkFVX...  2025-06-25 06:36:02  
1  https://news.google.com/rss/articles/CBMiVkFVX...  2025-06-23 05:13:16  
2  https://news.google.com/rss/articles/CBMiWkFVX...  2025-06-22 08:29:49  
3  https://news.google.com/rss/articles/CBMigwFBV...  2025-06-22 16:19:04  
4  https://news.google.com/rss/articles/CBMic0FVX...  2025-06-25 05:58:51  


### 감성 점수

In [40]:
import pandas as pd
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# ✅ 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("kluebert-sentiment")
tokenizer = AutoTokenizer.from_pretrained("kluebert-sentiment")
sentiment_pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=-1)

# ✅ 감성 점수 함수 정의
def get_sentiment_score(text):
    try:
        result = sentiment_pipe(text[:512])[0]  # 길이 제한
        return result["score"] if result["label"] == "LABEL_1" else 1 - result["score"]
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

# ✅ 뉴스 데이터 로딩
df_total_news = pd.read_csv("kospi200_news_last7days.csv")

# ✅ tqdm을 활용한 감성 점수 추론
sentiment_scores = []
print("🟡 감성 점수 추론 중...")

for idx, row in tqdm(df_total_news.iterrows(), total=len(df_total_news)):
    score = get_sentiment_score(row["title"])
    sentiment_scores.append(score)

# ✅ 결과 저장
df_total_news["sentiment"] = sentiment_scores

# ✅ 일부 결과 출력
print("\n✅ 상위 5개 감성 점수 결과:")
print(df_total_news[["keyword", "title", "sentiment"]].head())

# ✅ 중간 저장
df_total_news.to_csv("kospi200_news_with_sentiment.csv", index=False, encoding="utf-8-sig")
print("✅ 전체 감성 점수 결과 저장 완료!")

Device set to use cpu


🟡 감성 점수 추론 중...


100%|██████████| 14072/14072 [38:40<00:00,  6.06it/s]



✅ 상위 5개 감성 점수 결과:
  keyword                                              title  sentiment
0    삼성전자  삼성전자, '프로젝트 무한' 올레도스 공급망 삼성D로 이원화 검토 - 전자부품 전문...   0.723739
1    삼성전자                 삼성전자 美 테일러 팹 내년 초 양산라인 투자 - 지디넷코리아   0.651777
2    삼성전자            드디어 '삼성전자의 시간' 오나…'가격 급등' 이례적 현상 - 한국경제   0.787109
3    삼성전자            “美 정부, 삼성전자·하이닉스 中 공장에 장비 반입 제한” - 조선일보   0.318725
4    삼성전자  삼성전자 파운드리 퀄컴 2나노 AP 수주할 듯, 한진만 내년 파운드리 부활 신호탄 ...   0.702239
✅ 전체 감성 점수 결과 저장 완료!


In [41]:
df_total_news.head()

,keyword,title,link,published,sentiment
0,삼성전자,"삼성전자, '프로젝트 무한' 올레도스 공급망 삼성D로 이원화 검토 - 전자부품 전문...",https://news.google.com/rss/articles/CBMiZkFVX...,2025-06-25 06:36:02,0.723739
1,삼성전자,삼성전자 美 테일러 팹 내년 초 양산라인 투자 - 지디넷코리아,https://news.google.com/rss/articles/CBMiVkFVX...,2025-06-23 05:13:16,0.651777
2,삼성전자,드디어 '삼성전자의 시간' 오나…'가격 급등' 이례적 현상 - 한국경제,https://news.google.com/rss/articles/CBMiWkFVX...,2025-06-22 08:29:49,0.787109
3,삼성전자,"“美 정부, 삼성전자·하이닉스 中 공장에 장비 반입 제한” - 조선일보",https://news.google.com/rss/articles/CBMigwFBV...,2025-06-22 16:19:04,0.318725
4,삼성전자,"삼성전자 파운드리 퀄컴 2나노 AP 수주할 듯, 한진만 내년 파운드리 부활 신호탄 ...",https://news.google.com/rss/articles/CBMic0FVX...,2025-06-25 05:58:51,0.702239
